In [1]:
#default_exp rescore.feature_extractor

### feature_extractor module contains the mail functionalities to extract features from AlphaDeep predictions.

In [2]:
#export
import pandas as pd
import numpy as np

from alphadeep.utils import logging
from alphadeep.pretrained_models import ModelManager
from alphadeep.model.ms2 import calc_ms2_similarity
from alphadeep.mass_spec.match import PepSpecMatch
from alphabase.peptide.fragment import get_charged_frag_types
from alphabase.peptide.precursor import (
    refine_precursor_df
)
from alphabase.peptide.fragment import (
    concat_precursor_fragment_dataframes
)

from alphadeep.settings import global_settings
perc_settings = global_settings['percolator']

from alphadeep.rescore.fdr import calc_fdr_for_df

from tqdm import tqdm

def match_one_raw(
    psm_df_one_raw,
    ms2_file,
    ms2_file_type,
    frag_types_to_match,
    ms2_ppm, ms2_tol,
):
    """ Internal function """
    match = PepSpecMatch(
        charged_frag_types=frag_types_to_match
    )

    # (
    #     psm_df, fragment_mz_df, 
    #     matched_intensity_df, matched_mz_err_df
    # ) = 
    return match.match_ms2_one_raw(
        refine_precursor_df(psm_df_one_raw),
        ms2_file=ms2_file,
        ms2_file_type=ms2_file_type, 
        ppm=ms2_ppm, tol=ms2_tol,
    )

def get_ms2_features(
    psm_df, frag_types,
    predict_intensity_df,
    matched_intensity_df,
    matched_mass_err_df,
)->pd.DataFrame:
    """ Extract ms2 features from the given 
    predict_intensity_df and matched_intensity_df. It will add columns:
      cos: cosine similarity between predicted and matched fragments
      sa: spectral angle between predicted and matched fragments
      spc: Spearman's rank correlation between predicted and matched fragments.
      cos_bion: ...
      cos_yion: ...
      sa_bion: ...
      sa_yion: ...
      spc_bion: ...
      spc_yion: ...
      frag_ratio: # matched fragments / # total b&y fragments
      frag_ratio_bion: # matched b fragments / # total b fragments
      frag_ratio_yion: # matched y fragments / # total y fragments
    """
    used_frag_types = frag_types
    predict_intensity_df = predict_intensity_df[
        used_frag_types
    ]

    psm_df, ms2_metrics_df = calc_ms2_similarity(
        psm_df, predict_intensity_df, 
        matched_intensity_df,
        charged_frag_types=used_frag_types,
        metrics=['COS','SA','SPC'],
    )
    psm_df.rename(
        columns={
            'COS':'cos','SA':'sa','SPC':'spc'
        },
        inplace=True
    )

    b_frag_types = [
        _t for _t in used_frag_types 
        if _t.startswith('b')
    ]
    y_frag_types = [
        _t for _t in used_frag_types 
        if _t.startswith('y')
    ]

    frag_position_hits = matched_intensity_df[
        used_frag_types
    ].values.any(axis=1)
    frag_ratio_ion = []
    for start_idx, end_idx in psm_df[
        ['frag_start_idx','frag_end_idx']
    ].values:
        frag_ratio_ion.append(
            np.mean(frag_position_hits[start_idx:end_idx])
        )
    psm_df['frag_ratio'] = frag_ratio_ion

    if len(b_frag_types) > 0:
        psm_df, ms2_metrics_df = calc_ms2_similarity(
            psm_df, predict_intensity_df, 
            matched_intensity_df,
            charged_frag_types=b_frag_types,
            metrics=['COS','SA','SPC'],
        )
        psm_df.rename(
            columns={
                'COS':'cos_bion','SA':'sa_bion','SPC':'spc_bion'
            },
            inplace=True
        )
        frag_position_hits = matched_intensity_df[
            b_frag_types
        ].values.any(axis=1)
        
        frag_ratio_ion = []
        for start_idx, end_idx in psm_df[
            ['frag_start_idx','frag_end_idx']
        ].values:
            frag_ratio_ion.append(
                np.mean(frag_position_hits[start_idx:end_idx])
            )
        psm_df['frag_ratio_bion'] = frag_ratio_ion
    else:
        psm_df[['cos_bion','sa_bion','spc_bion']] = 0
        psm_df[['frag_ratio_bion']] = 0

    if len(y_frag_types) > 0:
        psm_df, ms2_metrics_df = calc_ms2_similarity(
            psm_df, predict_intensity_df, 
            matched_intensity_df,
            charged_frag_types=y_frag_types,
            metrics=['COS','SA','SPC'],
        )
        psm_df.rename(
            columns={
                'COS':'cos_yion','SA':'sa_yion','SPC':'spc_yion'
            },
            inplace=True
        )
        frag_position_hits = matched_intensity_df[
            y_frag_types
        ].values.any(axis=1)
        frag_ratio_ion = []
        for start_idx, end_idx in psm_df[
            ['frag_start_idx','frag_end_idx']
        ].values:
            frag_ratio_ion.append(
                np.mean(frag_position_hits[start_idx:end_idx])
            )
        psm_df['frag_ratio_yion'] = frag_ratio_ion
    else:
        psm_df[['cos_yion','sa_yion','spc_yion']] = 0
        psm_df[['frag_ratio_yion']] = 0

    return psm_df

# for imap/imap_unordered with multiprocessing.Pool()
def match_one_raw_mp(args):
    return match_one_raw(*args)
    
# for imap/imap_unordered with multiprocessing.Pool()
def get_ms2_features_mp(args):
    return get_ms2_features(*args)


### ScoreFeatureExtractor
`ScoreFeatureExtractor` and its sub-classes are the most important functionalities for rescoring.

They define the fine-tuning parameters to tune RT/CCS/MS2 models. They also define the feature list to feed to Percolator.

We recommend to use `ScoreFeatureExtractorMP` as it is faster by using multiprocessing while remain the prediction performance.

In [3]:
#export

class ScoreFeatureExtractor:
    def __init__(self, 
        model_mgr:ModelManager
    ):
        """ ScoreFeatureExtractor: Feature extractor for percolator 
              with a single process.

        Args:
            model_mgr (ModelManager): The ModelManager in alphadeep.pretrained_models.
        """
        self.model_mgr = model_mgr

        self.raw_num_to_tune = perc_settings['raw_num_to_tune']

        (
            self.model_mgr.psm_num_to_tune_ms2
        ) = perc_settings['psm_num_to_tune_ms2']

        (
            self.model_mgr.psm_num_per_mod_to_tune_ms2
        ) = perc_settings['psm_num_per_mod_to_tune_ms2']

        (
            self.model_mgr.psm_num_to_tune_rt_ccs
        ) = perc_settings['psm_num_to_tune_rt_ccs']

        (
            self.model_mgr.mod_psm_num_to_tune_rt_ccs
        ) = perc_settings['mod_psm_num_to_tune_rt_ccs']

        (
            self.model_mgr.top_n_mods_to_tune
        ) = perc_settings['top_n_mods_to_tune']

        self.require_model_tuning = perc_settings[
            'require_model_tuning'
        ]
        self.require_raw_specific_tuning = perc_settings[
            'require_raw_specific_tuning'
        ]
        self.raw_specific_ms2_tuning = perc_settings[
            'raw_specific_ms2_tuning'
        ]

        self.score_feature_list = [
            'cos','sa','spc',
            'cos_bion','sa_bion','spc_bion',
            'cos_yion','sa_yion','spc_yion',
            'frag_ratio','frag_ratio_bion',
            'frag_ratio_yion','rt_delta_abs', 
            'mobility_delta_abs',
        ]

    def _select_raw_to_tune(self,
        psm_df:pd.DataFrame,
    )->tuple:
        """ Randomly select `self.raw_num_to_tune` raw files 
        to tune the models. If # raw files is less than `self.raw_num_to_tune`,
        all raw files will be used to tune the model.

        Args:
            psm_df (pd.DataFrame): dataframe contains PSMs of all raw files.

        Returns:
            df_groupby_raw: psm_df.groupby('raw_name')
            list: selected raw_name list
        """
        if 'fdr' not in psm_df.columns:
            psm_df = calc_fdr_for_df(psm_df, 'score')
        df_fdr = psm_df[(psm_df.fdr<0.01)&(psm_df.decoy==0)]

        df_groupby_raw = df_fdr.groupby('raw_name')

        if df_groupby_raw.ngroups < self.raw_num_to_tune:
            tune_raw_num = df_groupby_raw.ngroups
        else:
            tune_raw_num = self.raw_num_to_tune

        raw_list = list(
            df_groupby_raw['score'].count().rank(
                ascending=True
            ).nlargest(tune_raw_num).index
        )

        return df_groupby_raw, raw_list
        

    def fine_tune_models(self,
        psm_df:pd.DataFrame,
        ms2_file_dict:dict,
        ms2_file_type:str,
        frag_types_to_match:str,
        ms2_ppm:bool, ms2_tol:float,
    ):
        """ Extract information from the ms2 files,
        and then fine-tune the models

        Args:
            psm_df (pd.DataFrame): psm_df
            ms2_file_dict (dict): {raw_name: ms2_file_path}
            ms2_file_type (str): ms2_file_type, could be 'alphapept', 'mgf', 'thermo_raw'
            frag_types_to_match (str): ['b_z1','b_z2','y_z1'...]
            ms2_ppm (bool): is ppm tolerance for ms2 matching
            ms2_tol (float): tolerance value for ms2 matching
        """
        logging.info('Preparing for fine-tuning ...')

        (
            df_groupby_raw, raw_list
        ) = self._select_raw_to_tune(psm_df)

        psm_df_list = []
        matched_intensity_df_list = []
        for raw_name, df in df_groupby_raw:
            if (
                raw_name not in raw_list 
                or raw_name not in ms2_file_dict
            ):
                continue
            (
                df, _, inten_df, _
            ) = match_one_raw(
                df, ms2_file_dict[raw_name],
                ms2_file_type,
                frag_types_to_match,
                ms2_ppm, ms2_tol,
            )
            psm_df_list.append(df)
            matched_intensity_df_list.append(inten_df)

        logging.info('Fine-tuning ...')
        self._tune(
            *concat_precursor_fragment_dataframes(
                psm_df_list,
                matched_intensity_df_list
            )
        )

    def _tune(self,
        psm_df, 
        matched_intensity_df
    ):
        if 'ccs' in psm_df.columns:
            self.model_mgr.fine_tune_ccs_model(psm_df)

        self.model_mgr.fine_tune_rt_model(psm_df)

        self.model_mgr.fine_tune_ms2_model(
            psm_df, matched_intensity_df
        )

    def extract_rt_features(self, psm_df):
        if self.require_raw_specific_tuning:
            (
                self.model_mgr.psm_num_to_tune_rt_ccs
            ) = perc_settings['psm_num_per_raw_to_tune']

            self.model_mgr.mod_psm_num_to_tune_rt_ccs = 0
            
            (
                self.model_mgr.epoch_to_tune_rt_ccs
            ) = perc_settings['epoch_per_raw_to_tune']
            self.model_mgr.fine_tune_rt_model(
                psm_df[(psm_df.fdr<0.01)&(psm_df.decoy==0)]
            )

        if 'rt_norm' in psm_df.columns:
            psm_df = self.model_mgr.predict_rt(
                psm_df
            )
            psm_df[
                'rt_delta'
            ] = (
                psm_df.rt_pred-psm_df.rt_norm
            )

            psm_df[
                'rt_delta_abs'
            ] = psm_df.rt_delta.abs()
        else:
            psm_df['rt_delta'] = 0
            psm_df['rt_delta_abs'] = 0

    def extract_mobility_features(self, psm_df):
        if (
            'mobility' in psm_df.columns
        ):
            psm_df = self.model_mgr.predict_mobility(
                psm_df
            )
            
            psm_df[
                'mobility_delta'
            ] = (
                psm_df.mobility_pred-psm_df.mobility
            )
            psm_df[
                psm_df.mobility_delta.isna(),'mobility_delta'
            ] = 0
            psm_df['mobility_delta_abs'] = psm_df.mobility_delta.abs()
        else:
            psm_df['mobility_delta'] = 0
            psm_df['mobility_delta_abs'] = 0
    

    def match_ms2(self,
        psm_df: pd.DataFrame,
        ms2_file_dict, #raw_name: ms2_file_path or ms_reader object
        ms2_file_type:str,
        frag_types_to_match:list = get_charged_frag_types(['b','y'], 2),
        ms2_ppm=True, ms2_tol=20,
    ):
        self.match = PepSpecMatch(
            charged_frag_types=frag_types_to_match
        )

        self.match.match_ms2_centroid(
            refine_precursor_df(psm_df),
            ms2_file_dict=ms2_file_dict,
            ms2_file_type=ms2_file_type, 
            ppm=ms2_ppm, tol=ms2_tol,
        )

    def extract_features(self,
        psm_df: pd.DataFrame,
        ms2_file_dict, 
        ms2_file_type, 
        frag_types:list = get_charged_frag_types(['b','y'], 2),
        ms2_ppm=True, ms2_tol=20,
    )->pd.DataFrame:
        """ Extract features and add columns (self.score_feature_list) into psm_df

        Args:
            psm_df (pd.DataFrame): psm dataframe to extract features
            ms2_file_dict ([type]): MS2 file path dict: {raw_name: ms2_path}
            ms2_file_type (str, optional): MS2 file type, coult be 
              'alphapept', 'mgf', or 'raw'.
            frag_types (list, optional): fragment types. 
              Defaults to `alphabase.fragment.get_charged_frag_types(['b','y'], 2)`.
            ms2_ppm (bool, optional): Matching MS2 mass tolerance unit. 
              Defaults to True.
            ms2_tol (int, optional): Matching mass tolerance. 
              Defaults to 20.

        Returns:
            pd.DataFrame: psm_df with feature columns added
        """
        
        frag_types = []
        for frag_type in frag_types:
            if frag_type in self.model_mgr.ms2_model.charged_frag_types:
                frag_types.append(frag_type)

        if self.require_model_tuning:
            logging.info('Fine-tuning models ...')
            self.fine_tune_models(
                psm_df, 
                ms2_file_dict, ms2_file_type,
                frag_types, ms2_ppm, ms2_tol
            )

        logging.info('Extracting alphadeep features ...')
        result_psm_list = []
        for raw_name, df in tqdm(psm_df.groupby('raw_name')):
            if raw_name not in ms2_file_dict:
                continue
            (
                df, frag_mz_df, frag_inten_df, frag_merr_df
            ) = match_one_raw(
                df,
                ms2_file_dict[raw_name],
                ms2_file_type,
                frag_types,
                ms2_ppm, ms2_tol,
            )

            self.extract_rt_features(df)
            self.extract_mobility_features(df)

            predict_inten_df = self.model_mgr.predict_ms2(df)

            result_psm_list.append(
                get_ms2_features(
                    df, frag_types, 
                    predict_inten_df,
                    frag_inten_df,
                    frag_merr_df,
                )
            )
            
        self.psm_df = pd.concat(
            result_psm_list
        ).reset_index(drop=True)
        logging.info('Finish extracting features')
        return self.psm_df
        

In [4]:
#hide
import torch
from copy import deepcopy

from alphadeep.model.ms2 import pDeepModel
from alphadeep.model.rt import AlphaRTModel
from alphadeep.model.ccs import AlphaCCSModel

def extract_rt_features(psm_df, model:torch.nn.Module):
    if 'rt_norm' in psm_df.columns:
        rt_model = AlphaRTModel()
        rt_model.model = deepcopy(model)
        psm_df = rt_model.predict(
            psm_df
        )
        psm_df[
            'rt_delta'
        ] = (
            psm_df.rt_pred-psm_df.rt_norm
        )

        psm_df[
            'rt_delta_abs'
        ] = psm_df.rt_delta.abs()
    else:
        psm_df['rt_delta'] = 0
        psm_df['rt_delta_abs'] = 0

def extract_mobility_features(psm_df, model:torch.nn.Module):
    if (
        'mobility' in psm_df.columns
    ):
        ccs_model = AlphaCCSModel()
        ccs_model.model = deepcopy(model)
        psm_df = ccs_model.predict(
            psm_df
        )

        psm_df = ccs_model.ccs_to_mobility_pred(
            psm_df
        )
        
        psm_df[
            'mobility_delta'
        ] = (
            psm_df.mobility_pred-psm_df.mobility
        )
        psm_df[
            psm_df.mobility_delta.isna(),'mobility_delta'
        ] = 0
        psm_df['mobility_delta_abs'] = psm_df.mobility_delta.abs()
    else:
        psm_df['mobility_delta'] = 0
        psm_df['mobility_delta_abs'] = 0

def predict_ms2(psm_df, 
    model:torch.nn.Module, 
    frag_types,
):
    pdeep = pDeepModel(frag_types)
    pdeep.model = deepcopy(model)
    return pdeep.predict(psm_df, batch_size=256)

### ScoreFeatureExtractorMP

`ScoreFeatureExtractorMP` uses multiprocessing to accelerate the feature extraction procedure. The pipeline is:
1. Randomly select some raw files to fine-tuning the model. This step also includes two steps: a. ms2 matching with multiprocessing; b. model tuning with a single thread (or GPU). 
2. Match PSM results of each raw file in the psm_df against the corresponding ms2 file in ms2_file_dict ({`raw_name`: `ms2_file_path`}) to get matched fragment intensity with multiprocessing.
3. Predict fragment intensity as well as RT and mobility with a single thread (or GPU). We use a single thread here becauase GPU memory is the main limitation, and we also enable raw-specific fine-tuning for different raw files.
4. Calculate the feature values with multiprocessing.

The key in `ScoreFeatureExtractorMP` is to access the GPU section without multiprocessing to avoid GPU memory conflicts.

The processing speed is very fast with a normal GPU (GTX1080). In our testing, AlphaDeep can rescore 371 HLA raw files within 1 hour.

In [5]:
#export
import torch
import torch.multiprocessing as mp

# from alphatims
def process_bar(iterator, len_iter):
    with tqdm(total=len_iter) as bar:
        for i,iter in enumerate(iterator):
            yield iter
            bar.update()
        bar.update(len_iter-i-1)
            

class ScoreFeatureExtractorMP(ScoreFeatureExtractor):
    def __init__(self, 
        model_mgr:ModelManager
    ):
        """ ScoreFeatureExtractorMP: Feature extractor for percolator 
              with multiprocessing.

        Args:
            model_mgr (ModelManager): The ModelManager in alphadeep.pretrained_models.
        """
        super().__init__(model_mgr=model_mgr)

        # share_memory to save memory
        self.model_mgr.ms2_model.model.share_memory()
        self.model_mgr.rt_model.model.share_memory()
        self.model_mgr.ccs_model.model.share_memory()


    def fine_tune_models(self,
        psm_df,
        ms2_file_dict,
        ms2_file_type,
        frag_types_to_match,
        ms2_ppm, ms2_tol,
    ):
        """ Extract information from the ms2 files,
        and then fine-tune the models

        Args:
            psm_df (pd.DataFrame): psm_df
            ms2_file_dict (dict): {raw_name: ms2_file_path}
            ms2_file_type (str): ms2_file_type, could be 'alphapept', 'mgf', 'thermo_raw'
            frag_types_to_match (str): ['b_z1','b_z2','y_z1'...]
            ms2_ppm (bool): is ppm tolerance for ms2 matching
            ms2_tol (float): tolerance value for ms2 matching
        """
        (
            df_groupby_raw, raw_list
        ) = self._select_raw_to_tune(psm_df)

        def one_raw_param_generator(df_groupby_raw):
            for raw_name, df in df_groupby_raw:
                if (
                    raw_name not in raw_list 
                    or raw_name not in ms2_file_dict
                ):
                    continue

                yield (
                    df,
                    ms2_file_dict[raw_name],
                    ms2_file_type,
                    frag_types_to_match,
                    ms2_ppm, ms2_tol,
                )
        
        logging.info('Preparing for fine-tuning ...')  
        psm_df_list = []
        matched_intensity_df_list = []  
        with mp.Pool(global_settings['thread_num']) as p:
            for df, _, inten_df, _ in process_bar(
                p.imap_unordered(
                    match_one_raw_mp, 
                    one_raw_param_generator(df_groupby_raw)
                ), df_groupby_raw.ngroups
            ):
                psm_df_list.append(df)
                matched_intensity_df_list.append(inten_df)

        logging.info('Fine-tuning ...')
        self._tune(
            *concat_precursor_fragment_dataframes(
                psm_df_list,
                matched_intensity_df_list
            )
        )

    def extract_features_one_raw(self,
        df_one_raw: pd.DataFrame,
        ms2_file, 
        ms2_file_type,
        frag_types,
        ms2_ppm, ms2_tol,
    ):
        (
            df, frag_mz_df, frag_inten_df, frag_merr_df
        ) = match_one_raw(df_one_raw, 
            ms2_file, ms2_file_type, frag_types,
            ms2_ppm, ms2_tol
        )

        self.extract_rt_features(df)
        self.extract_mobility_features(df)

        predict_inten_df = self.model_mgr.predict_ms2(df)

        return get_ms2_features(df, 
            frag_types, 
            predict_inten_df,
            frag_inten_df,
            frag_merr_df,
        )

    def extract_features(self,
        psm_df: pd.DataFrame,
        ms2_file_dict,
        ms2_file_type,
        frag_types:list = get_charged_frag_types(['b','y'], 2),
        ms2_ppm=True, ms2_tol=20,
    )->pd.DataFrame:

        """ MPExtract features and add columns (self.score_feature_list) into psm_df.
        Main entry of ScoreFeatureExtractor.

        Args:
            psm_df (pd.DataFrame): psm dataframe to extract features
            ms2_file_dict ([type]): MS2 file path dict: {raw_name: ms2_path}
            ms2_file_type (str, optional): MS2 file type, coult be 
              'alphapept', 'mgf', or 'thermo'.
            frag_types (list, optional): fragment types. 
              Defaults to `alphabase.fragment.get_charged_frag_types(['b','y'], 2)`.
            ms2_ppm (bool, optional): Matching MS2 mass tolerance unit. 
              Defaults to True.
            ms2_tol (int, optional): Matching mass tolerance. 
              Defaults to 20.

        Returns:
            pd.DataFrame: psm_df with feature columns added
        """

        used_frag_types = []
        for frag_type in frag_types:
            if frag_type in self.model_mgr.ms2_model.charged_frag_types:
                used_frag_types.append(frag_type)

        if self.require_model_tuning:
            logging.info('Fine-tuning models ...')
            self.fine_tune_models(
                psm_df, 
                ms2_file_dict, ms2_file_type,
                used_frag_types, ms2_ppm, ms2_tol
            )

        def one_raw_param_generator(df_groupby_raw):
            for raw_name, df in df_groupby_raw:
                if raw_name not in ms2_file_dict:
                    continue
                yield (
                    df,
                    ms2_file_dict[raw_name],
                    ms2_file_type,
                    used_frag_types,
                    ms2_ppm, ms2_tol,
                )

        logging.info(
            'Extracting alphadeep features with multiprocessing ...'
        )
        df_groupby_raw = psm_df.groupby('raw_name')
        result_psm_list = []
        if (
            not torch.cuda.is_available() and 
            not self.require_raw_specific_tuning
        ):
            # use multiprocessing for prediction 
            # only when no GPUs are available
            with mp.Pool(global_settings['thread_num']) as p:
                for _df in process_bar(p.imap_unordered(
                    self.extract_features_one_raw, 
                    one_raw_param_generator(df_groupby_raw)
                ), df_groupby_raw.ngroups):
                    result_psm_list.append(_df)

        else:
            # multiprocessing is only used for ms2 matching
            def prediction_gen(df_groupby_raw):
                with mp.Pool(global_settings['thread_num']) as _p:
                    for (
                        df, frag_mz_df, frag_inten_df, frag_merr_df
                    ) in _p.imap_unordered(
                        match_one_raw_mp, 
                        one_raw_param_generator(df_groupby_raw)
                    ):
                        # outsite multiprocessing region
                        self.extract_rt_features(df)
                        self.extract_mobility_features(df)

                        if (
                            self.require_raw_specific_tuning 
                            and self.raw_specific_ms2_tuning
                        ):
                            (
                                self.model_mgr.psm_num_to_tune_ms2
                            ) = perc_settings['psm_num_per_raw_to_tune']

                            self.model_mgr.psm_num_per_mod_to_tune_ms2 = 0

                            self.model_mgr.epoch_to_tune_ms2 = 3

                            self.model_mgr.grid_nce_search = False

                            if 'nce' not in df.columns:
                                self.model_mgr.set_default_nce(df)

                            self.model_mgr.fine_tune_ms2_model(
                                df[(df.fdr<0.01)&(df.decoy==0)],
                                frag_inten_df
                            )
                            
                        predict_inten_df = self.model_mgr.predict_ms2(df)

                        yield (
                            df, used_frag_types, 
                            predict_inten_df,
                            frag_inten_df, frag_merr_df,
                        )

            with mp.Pool(global_settings['thread_num']) as p:
                for df in process_bar(p.imap_unordered(
                    get_ms2_features_mp, 
                    prediction_gen(df_groupby_raw)
                ), df_groupby_raw.ngroups):
                    result_psm_list.append(df)
                    
        self.psm_df = pd.concat(
            result_psm_list
        ).reset_index(drop=True)
        logging.info('Finished feature extraction with multiprocessing')
        return self.psm_df

In [7]:
#hide
from alphadeep.pretrained_models import ModelManager
model_mgr = ModelManager()
model_mgr.load_installed_models()
ScoreFeatureExtractorMP(model_mgr)
ScoreFeatureExtractor(model_mgr)